In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Le Port,RE,-20.9373,55.2919,67.24,71,4,3.33,light rain
1,1,Kodiak,US,57.7900,-152.4072,51.31,82,100,12.66,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,87,20,19.57,few clouds
3,3,Tiksi,RU,71.6872,128.8694,34.70,76,100,17.54,overcast clouds
4,4,Butaritari,KI,3.0707,172.7902,81.81,77,83,14.63,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Le Port,RE,-20.9373,55.2919,67.24,71,4,3.33,light rain
1,1,Kodiak,US,57.7900,-152.4072,51.31,82,100,12.66,overcast clouds
5,5,Severo-Kurilsk,RU,50.6789,156.1250,47.86,94,100,1.25,overcast clouds
6,6,Barcelos,PT,41.5388,-8.6151,60.91,86,52,7.25,broken clouds
7,7,Bredasdorp,ZA,-34.5322,20.0403,49.21,85,10,3.60,clear sky
9,9,Tasiilaq,GL,65.6145,-37.6368,44.76,68,64,4.14,broken clouds
10,10,Hobart,AU,-42.8794,147.3294,53.26,83,75,6.91,broken clouds
11,11,Mataura,NZ,-46.1927,168.8643,42.48,92,95,1.10,overcast clouds
12,12,Arica,CL,-18.4750,-70.3042,62.15,77,0,4.61,clear sky
13,13,Rosetown,CA,51.5501,-108.0014,69.66,82,100,10.36,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

Unnamed: 0             359
City                   359
Country                353
Lat                    359
Lng                    359
Max Temp               359
Humidity               359
Cloudiness             359
Wind Speed             359
Current Description    359
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Le Port,RE,67.24,light rain,-20.9373,55.2919,
1,Kodiak,US,51.31,overcast clouds,57.7900,-152.4072,
5,Severo-Kurilsk,RU,47.86,overcast clouds,50.6789,156.1250,
6,Barcelos,PT,60.91,broken clouds,41.5388,-8.6151,
7,Bredasdorp,ZA,49.21,clear sky,-34.5322,20.0403,
9,Tasiilaq,GL,44.76,broken clouds,65.6145,-37.6368,
10,Hobart,AU,53.26,broken clouds,-42.8794,147.3294,
11,Mataura,NZ,42.48,overcast clouds,-46.1927,168.8643,
12,Arica,CL,62.15,clear sky,-18.4750,-70.3042,
13,Rosetown,CA,69.66,heavy intensity rain,51.5501,-108.0014,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame

    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna(axis="index", how="any")

#clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Le Port,RE,67.24,light rain,-20.9373,55.2919,KORBEY D'OR OUEST
1,Kodiak,US,51.31,overcast clouds,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
5,Severo-Kurilsk,RU,47.86,overcast clouds,50.6789,156.1250,Kak Doma
6,Barcelos,PT,60.91,broken clouds,41.5388,-8.6151,Hotel Restaurante Bagoeira
7,Bredasdorp,ZA,49.21,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
9,Tasiilaq,GL,44.76,broken clouds,65.6145,-37.6368,The Red House
10,Hobart,AU,53.26,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
11,Mataura,NZ,42.48,overcast clouds,-46.1927,168.8643,Ellie's Villa
12,Arica,CL,62.15,clear sky,-18.4750,-70.3042,Hostal Jardin Del Sol
13,Rosetown,CA,69.66,heavy intensity rain,51.5501,-108.0014,Travelodge by Wyndham Rosetown


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index=False)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))